## Imports and definition of leagues

In [192]:
# load modules, install if not present
try:
    import pandas as pd
except ImportError:
    !pip install pandas
    import pandas as pd

try:
    import numpy as np
except ImportError:
    !pip install numpy
    import numpy as np

try:
    import requests
except ImportError:
    !pip install requests
    import requests

try: 
  import sqlite3 as db
except ImportError:
  !pip install sqlite3
  import sqlite3 as db

# already included in python
import io, os

In [193]:
# define league IDs
league_dic = {"Women's Hockey League": 'WL', '1. Liga': 'L1', 'U-21 Elit': 'U21', 'National League': 'NL', 'Swiss League': 'SL', 'MyHockey League': 'ML'}

## Download functions

In [194]:
# Gets the URL for game-CSV for given leagues and seasons
def games_url(year_of_past_games, league_id):
  date_range = '01.09.' + str(year_of_past_games-1) + '-31.08.' + str(year_of_past_games)
  if league_id == 'WL':
    return f'https://data.sihf.ch/Statistic/api/cms/export?alias=results&searchQuery=1/2015-2099/42&filterQuery={str(year_of_past_games)}/all/{date_range}//all/all&orderBy=date&orderByDescending=false&filterBy=season,phase,date,deferredState,team1,team2&format=csv'
  elif league_id == 'U21' and year_of_past_games >= 2026:
    return f'https://data.sihf.ch/Statistic/api/cms/export?alias=results&searchQuery=1,10,11/2015-2099/4,5,14,15,16,23,24,25,26,28,27,29,30,31,32,60,61,105,106,107,113,114,115,116,117,118,119,120,121,122,123,124,125&filterQuery={str(year_of_past_games)}/113/all/all/{date_range}//all/all&orderBy=date&orderByDescending=false&filterBy=season,league,region,phase,date,deferredState,team1,team2&format=csv'
  elif league_id == 'U21' and year_of_past_games < 2026:
    return f'https://data.sihf.ch/Statistic/api/cms/export?alias=results&searchQuery=1,10,11/2015-2099/4,5,14,15,16,23,24,25,26,28,27,29,30,31,32,60,61,105,106,107,113,114,115,116,117,118,119,120,121,122,123,124,125&filterQuery={str(year_of_past_games)}/4/all/all/{date_range}//all/all&orderBy=date&orderByDescending=false&filterBy=season,league,region,phase,date,deferredState,team1,team2&format=csv'
  elif league_id == 'L1':
    return f'https://data.sihf.ch/Statistic/api/cms/export?alias=results&searchQuery=1,10/2015-2099/3,10,18,19,33,35,36,38,37,39,40,41,43,101,44,45,46,104,83&filterQuery={str(year_of_past_games)}/3/all/all/{date_range}//all/all&orderBy=date&orderByDescending=false&filterBy=season,league,region,phase,date,deferredState,team1,team2&format=csv'
  elif league_id == 'NL':
    return f'https://data.sihf.ch/Statistic/api/cms/export?alias=results&searchQuery=1,10//1&filterQuery={str(year_of_past_games)}/all/{date_range}//all/all&filterBy=season,phase,date,deferredState,team1,team2&orderBy=date&orderByDescending=false&format=csv'
  elif league_id == 'SL':
    return f'https://data.sihf.ch/Statistic/api/cms/export?alias=results&searchQuery=1,10//2&filterQuery={str(year_of_past_games)}/all/{date_range}//all/all&filterBy=season,phase,date,deferredState,team1,team2&orderBy=date&orderByDescending=false&format=csv'
  elif league_id == 'ML':
    return f'https://data.sihf.ch/Statistic/api/cms/export?alias=results&searchQuery=1,8,10/2018-2099/100&filterQuery={str(year_of_past_games)}/all/{date_range}//all/all&filterBy=season,phase,date,deferredState,team1,team2&orderBy=date&orderByDescending=false&format=csv'
  else:
     return 'Could not find csv-URL for this League ID: ' + league_id

# Gets the URL for game-CSV for given leagues and seasons
def season_url(year_of_past_games, league_id):
  if league_id == 'WL' and year_of_past_games <= 2021:
    return f'https://data.sihf.ch/Statistic/api/cms/export?alias=standing&searchQuery=1,10/2015-2099/42&filterQuery={str(year_of_past_games)}/2767&orderBy=rank&orderByDescending=false&filterBy=Season&format=csv'
  elif league_id == 'WL' and year_of_past_games == 2022:
    return f'https://data.sihf.ch/Statistic/api/cms/export?alias=standing&searchQuery=1,10/2015-2099/42&filterQuery={str(year_of_past_games)}/3573&orderBy=rank&orderByDescending=false&filterBy=Season,Phase&format=csv'
  elif league_id == 'WL' and year_of_past_games == 2023:
    return f'https://data.sihf.ch/Statistic/api/cms/export?alias=standing&searchQuery=1,10/2015-2099/42&filterQuery={str(year_of_past_games)}/3844&orderBy=rank&orderByDescending=false&filterBy=Season,Phase&format=csv'
  elif league_id == 'WL' and year_of_past_games > 2023:
    return f'https://data.sihf.ch/Statistic/api/cms/export?alias=standing&searchQuery=1,10/2015-2099/42&filterQuery={str(year_of_past_games)}/4261&orderBy=rank&orderByDescending=false&filterBy=Season,Phase&format=csv'
  elif league_id == 'U21' and year_of_past_games >= 2025:
    return f'https://data.sihf.ch/Statistic/api/cms/export?alias=standing&searchQuery=1,10,11/2015-2099/4,5,14,15,16,23,24,25,26,28,27,29,30,31,32,60,61,105,106,107,113,114,115,116,117,118,119,120,121,122,123,124,125&filterQuery={str(year_of_past_games)}/4/1/4688/All&orderBy=rank&orderByDescending=false&filterBy=Season,League,Region,Phase,ContentType&format=csv'
  elif league_id == 'U21' and year_of_past_games == 2024:
    return f'https://data.sihf.ch/Statistic/api/cms/export?alias=standing&searchQuery=1,10,11/2015-2099/4,5,14,15,16,23,24,25,26,28,27,29,30,31,32,60,61,105,106,107,113,114,115,116,117,118,119,120,121,122,123,124,125&filterQuery={str(year_of_past_games)}/4/1/4326/All&orderBy=rank&orderByDescending=false&filterBy=Season,League,Region,Phase,ContentType&format=csv'
  elif league_id == 'U21' and year_of_past_games == 2023:
    return f'https://data.sihf.ch/Statistic/api/cms/export?alias=standing&searchQuery=1,10,11/2015-2099/4,5,14,15,16,23,24,25,26,28,27,29,30,31,32,60,61,105,106,107,113,114,115,116,117,118,119,120,121,122,123,124,125&filterQuery={str(year_of_past_games)}/4/1/3967/All&orderBy=rank&orderByDescending=false&filterBy=Season,League,Region,Phase,ContentType&format=csv'
  elif league_id == 'U21' and year_of_past_games in [2022,2021]:
    return f'https://data.sihf.ch/Statistic/api/cms/export?alias=standing&searchQuery=1,10,11/2015-2099/4,5,14,15,16,23,24,25,26,28,27,29,30,31,32,60,61,105,106,107,113,114,115,116,117,118,119,120,121,122,123,124,125&filterQuery={str(year_of_past_games)}/4/1/3645/All&orderBy=rank&orderByDescending=false&filterBy=Season,League,Region,Phase,ContentType&format=csv'
  elif league_id == 'U21' and year_of_past_games == 2020:
    return f'https://data.sihf.ch/Statistic/api/cms/export?alias=standing&searchQuery=1,10,11/2015-2099/4,5,14,15,16,23,24,25,26,28,27,29,30,31,32,60,61,105,106,107,113,114,115,116,117,118,119,120,121,122,123,124,125&filterQuery={str(year_of_past_games)}/4/1/3176/All&orderBy=rank&orderByDescending=false&filterBy=Season,League,Region,Phase,ContentType&format=csv'
  elif league_id == 'U21' and year_of_past_games < 2020:
    return f'https://data.sihf.ch/Statistic/api/cms/export?alias=standing&searchQuery=1,10,11/2015-2099/4,5,14,15,16,23,24,25,26,28,27,29,30,31,32,60,61,105,106,107,113,114,115,116,117,118,119,120,121,122,123,124,125&filterQuery={str(year_of_past_games)}/4/1/2829/All&orderBy=rank&orderByDescending=false&filterBy=Season,League,Region,Phase,ContentType&format=csv'
  elif league_id == 'L1':
    if year_of_past_games == 2026:
      fillin = ['4969', '4968']
    elif year_of_past_games == 2025:
      fillin = ['4626', '4625']
    elif year_of_past_games == 2024:
      fillin = ['4231', '4187']
    elif year_of_past_games == 2023:
      fillin = ['3803', '3804']
    elif year_of_past_games == 2022:
      fillin = ['3515', '3514']
    elif year_of_past_games == 2021:
      fillin = ['3354', '3352']
    elif year_of_past_games == 2020:
      fillin = ['3037', '3036']
    elif year_of_past_games == 2019:
      fillin = ['2710', '2709']
    return [f'https://data.sihf.ch/Statistic/api/cms/export?alias=standing&searchQuery=1,10/2015-2099/3,10,18,19,33,35,36,38,37,39,40,41,43,101,44,45,46,104&filterQuery={str(year_of_past_games)}/3/5/{fillin[0]}&orderBy=rank&orderByDescending=false&filterBy=Season,League,Region,Phase&format=csv',
            f'https://data.sihf.ch/Statistic/api/cms/export?alias=standing&searchQuery=1,10/2015-2099/3,10,18,19,33,35,36,38,37,39,40,41,43,101,44,45,46,104&filterQuery={str(year_of_past_games)}/3/5/{fillin[1]}&orderBy=rank&orderByDescending=false&filterBy=Season,League,Region,Phase&format=csv']
  elif league_id == 'NL' and year_of_past_games != 2020:
    return f'https://data.sihf.ch/Statistic/api/cms/export?alias=standing&searchQuery=1//1&filterQuery={str(year_of_past_games)}/2823/All&filterBy=Season,Phase,ContentType&orderBy=rank&orderByDescending=false&format=csv'
  elif league_id == 'NL' and year_of_past_games == 2020:
    return f'https://data.sihf.ch/Statistic/api/cms/export?alias=standing&searchQuery=1//1&filterQuery={str(year_of_past_games)}/3092/All&filterBy=Season,Phase,ContentType&orderBy=rank&orderByDescending=false&format=csv'
  elif league_id == 'SL' and year_of_past_games >= 2021:
    return f'https://data.sihf.ch/Statistic/api/cms/export?alias=standing&searchQuery=1//2&filterQuery={str(year_of_past_games)}/4941/All&filterBy=Season,Phase,ContentType&orderBy=rank&orderByDescending=false&format=csv'
  elif league_id == 'SL' and year_of_past_games < 2021:
    return f'https://data.sihf.ch/Statistic/api/cms/export?alias=standing&searchQuery=1//2&filterQuery={str(year_of_past_games)}/3174/All&filterBy=Season,Phase,ContentType&orderBy=rank&orderByDescending=false&format=csv'
  elif league_id == 'ML' and year_of_past_games >= 2024:
    return f'https://data.sihf.ch/Statistic/api/cms/export?alias=standing&searchQuery=1/2018-2099/100&filterQuery={str(year_of_past_games)}/4948/All&filterBy=Season,Phase,ContentType&orderBy=rank&orderByDescending=false&format=csv'
  elif league_id == 'ML' and year_of_past_games == 2023:
    return f'https://data.sihf.ch/Statistic/api/cms/export?alias=standing&searchQuery=1/2018-2099/100&filterQuery={str(year_of_past_games)}/3813/All&filterBy=Season,Phase,ContentType&orderBy=rank&orderByDescending=false&format=csv'
  elif league_id == 'ML' and year_of_past_games in [2022,2021]:
    return f'https://data.sihf.ch/Statistic/api/cms/export?alias=standing&searchQuery=1/2018-2099/100&filterQuery={str(year_of_past_games)}/3544/All&filterBy=Season,Phase,ContentType&orderBy=rank&orderByDescending=false&format=csv'
  elif league_id == 'ML' and year_of_past_games == 2020:
    return f'https://data.sihf.ch/Statistic/api/cms/export?alias=standing&searchQuery=1/2018-2099/100&filterQuery={str(year_of_past_games)}/3072/All&filterBy=Season,Phase,ContentType&orderBy=rank&orderByDescending=false&format=csv'
  elif league_id == 'ML' and year_of_past_games < 2020:
    return f'https://data.sihf.ch/Statistic/api/cms/export?alias=standing&searchQuery=1/2018-2099/100&filterQuery={str(year_of_past_games)}/2812/All&filterBy=Season,Phase,ContentType&orderBy=rank&orderByDescending=false&format=csv'
  else:
     return 'Could not find csv-URL for this League ID: ' + league_id

# Gets stats for games, players and goalies for a given game-ID
def game_url(game_id):
  game_url_dic = {'player_home': f'https://data.sihf.ch/statistic/api/cms/export/gamestats?searchQuery={game_id}&language=de&alias=gamePlayerStatsHome&format=csv',
                  'goalie_home': f'https://data.sihf.ch/statistic/api/cms/export/gamestats?searchQuery={game_id}&language=de&alias=gameGoalieStatsHome&format=csv',
                  'player_away': f'https://data.sihf.ch/statistic/api/cms/export/gamestats?searchQuery={game_id}&language=de&alias=gamePlayerStatsAway&format=csv',
                  'goalie_away': f'https://data.sihf.ch/statistic/api/cms/export/gamestats?searchQuery={game_id}&language=de&alias=gameGoalieStatsAway&format=csv',
                  'game': f'https://data.sihf.ch/statistic/api/cms/export/gamestats?searchQuery={game_id}&language=de&alias=gameTeamStats&format=csv'}
  return game_url_dic

In [195]:
# downloads the game csv-data-tables and returns as pandas dataframe
def get_table(url, league_id, year):
  headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
        "Accept-Encoding": "gzip, deflate, br"}
  try:
      response = requests.get(url, headers=headers)
      # Check if the request was successful
      if response.status_code == 200:
          return pd.read_csv(io.StringIO(response.text), sep = ';')
      else:
          print(f"Failed to retrieve the table for {league_id} in year {year}. Status code: {response.status_code}")
  except Exception as e:
      print(f"An error occurred: {e}")

# Downloads

## Get all games for given years

In [ ]:
# Get the general game info for all leagues and seasons
games_dic = {}
for league_name, league_code in league_dic.items():
    games = {}
    for year in [2019,2020,2021,2022,2023,2024,2025,2026]:
        url = games_url(year_of_past_games=year, league_id=league_code)
        dat = get_table(url, league_id=league_code, year=year)
        games[str(year)] = dat
    games = pd.concat(games, axis=0).reset_index()
    games_dic[league_code] = games

## Get all season results

In [ ]:
# Get the general game info for all leagues and seasons
tournament_dic = {}
for league_name, league_code in league_dic.items():
    games = {}
    for year in [2019,2020,2021,2022,2023,2024,2025,2026]:
        if league_code == 'L1':
            url = season_url(year_of_past_games=year, league_id=league_code)
            dat = get_table(url[0], league_id=league_code, year=year)
            dat['Gruppe'] = 'Ost'
            dat2 = get_table(url[1], league_id=league_code, year=year)
            dat2['Gruppe'] = 'West'
            dat = pd.concat([dat, dat2], axis=0).reset_index()
        else:
            url = season_url(year_of_past_games=year, league_id=league_code)
            dat = get_table(url, league_id=league_code, year=year)
    games[str(year)] = dat
    games = pd.concat(games, axis=0).reset_index()
    tournament_dic[league_code] = games

## Get info on each player in each game

In [200]:
# Get the player and goalie infos
player_dic = {}
goalie_dic = {}

for league_name, league_code in league_dic.items():
    games = games_dic[league_code]
    game_goalies = {}
    game_players = {}
    for i, row in games.iterrows():
        game_id = row['Id']
        if row['Resultat'].strip() != '-:-':
            urls = game_url(game_id)
            goalies = {}
            players = {}
            for key, url in urls.items():
                dat = get_table(url, league_code, row['level_0'])
                if key.startswith('goalie'):
                    goalies[key] = dat
                elif key.startswith('player'):
                    players[key] = dat
                else:
                    'Cound not match table key'
            goalies = pd.concat(goalies, axis=0).reset_index()
            goalies['game_id'] = game_id
            players = pd.concat(players, axis=0).reset_index()
            players['game_id'] = game_id
            game_goalies[game_id] = goalies
            game_players[game_id] = players
    # Merge data
    game_goalies = pd.concat(game_goalies, axis=0).reset_index(drop=True)
    game_players = pd.concat(game_players, axis=0).reset_index(drop=True)
    game_players = game_players[game_players['Spieler'] != 'Total']
    player_dic[league_code] = game_players
    goalie_dic[league_code] = game_goalies


## Get game stats

In [239]:
# Fetch the game details
game_stats_dic = {}
for league_name, league_code in league_dic.items():
    games = games_dic[league_code]
    game_stats = {}
    for i, row in games.iterrows():
        game_id = row['Id']
        if row['Resultat'].strip() != '-:-':
            urls = game_url(game_id)
            dat = get_table(urls['game'], league_code, row['level_0'])
            dat['game_id'] = game_id
            home = dat.columns[1]
            away = dat.columns[2]
            dat.rename(columns={home: 'Home', away: 'Away'}, inplace=True)
            dat.loc[len(dat)] = ['Teams', home, away, game_id]
            game_stats[game_id] = dat
    game_stats = pd.concat(game_stats, axis=0).reset_index()
    game_stats_dic[league_code] = game_stats


# Compile an save tables

## Teams

In [ ]:
# get table of teams, standardize team names and add logo images
for league_name, league_code in league_dic.items():
    games = games_dic[league_code]
    teams = set(games['Home'].unique()).union(set(games['Away'].unique()))
    teams=pd.DataFrame(list(teams),columns=['team_name'])
    teams['team_id']=teams.index + 1
    logos = pd.read_excel('../data/4321-manual.xlsx')
    logos['png_url'] = 'https://www.sihf.ch' + logos['png_url']
    teams = pd.merge(teams, logos, on='team_name', how='left')
    #teams = teams[['team_id', 'team_name', 'png_url']]
    teams['team_id'] = league_code + '_' + teams['team_id'].astype(str)
    teams['png_path'] = teams['png_url'].str.replace('https://www.sihf.ch/Image/Club/', f'../data/images/{league_code}_logos/')
    # Download and save team logos
    for i, row in teams.iterrows():
        #print(row['png_url'])
        img_data = requests.get(row['png_url']).content
        os.makedirs(os.path.dirname(row['png_path']), exist_ok=True)
        with open(row['png_path'], 'wb') as handler:
            handler.write(img_data)
    # Save data to SQLite database
    conn = db.connect(f'../data/hockey_stats.db')
    teams.to_sql(f'{league_code}_teams', conn, if_exists='replace', index=False)
    conn.commit()
    conn.close()


## Players

In [242]:
# compile and save table of players
for league_name, league_code in league_dic.items():
    players = set(player_dic[league_code]['Spieler'].unique())
    players=pd.DataFrame(list(players),columns=['player_name'])
    players['player_id']=players.index + 1
    players['player_id'] = league_code + '_' + players['player_id'].astype(str)
    players = players[['player_id', 'player_name']]
    # Save data to SQLite database
    conn = db.connect(f'../data/hockey_stats.db')
    players.to_sql(f'{league_code}_players', conn, if_exists='replace', index=False)
    conn.commit()
    conn.close()

## Goalies

In [243]:
# get tables of goalies
for league_name, league_code in league_dic.items():
    goalies = set(goalie_dic[league_code]['Spieler'].unique())
    goalies=pd.DataFrame(list(goalies),columns=['player_name'])
    goalies['player_id']=goalies.index + 1
    goalies['player_id'] = league_code + '_' + goalies['player_id'].astype(str)
    goalies = goalies[['player_id', 'player_name']]
    # Save data to SQLite database
    conn = db.connect('../data/hockey_stats.db')
    goalies.to_sql(f'{league_code}_goalies', conn, if_exists='replace', index=False)
    conn.commit()
    conn.close()

## Games

In [244]:
# Initialize a game data dataframe
df_columns = ["id", "Season", "Region", "Phase", "Date", "Weekday", "Time", "Spectators", "Home_Team", "Away_Team", "Result", "Period_Results",
              "OT", "SO", "Home_Goals_Forwards", "Away_Goals_Forwards", "Home_Goals_Defense", "Away_Goals_Defense", 
              "Home_Assists_Forwards", "Away_Assists_Forwards", "Home_Assists_Defense", "Away_Assists_Defense", "Home_Penalty_Min", "Away_Penalty_Min",
              "Home_Powerplay_Opportunities", "Away_Powerplay_Opportunities", "Home_Powerplay_Goals", "Away_Powerplay_Goals", "Home_Shorthanded_Goals",
              "Away_Shorthanded_Goals", "Home_Powerplay_Min", "Away_Powerplay_Min"]

for league_name, league_code in league_dic.items():
    game_info = pd.DataFrame(columns=df_columns)
    games = games_dic[league_code]
    # Loop through each game and extract the relevant stats
    for i, row in games.iterrows():
        game_id = row['Id']
        # only process games that have been played
        if row['Resultat'].strip() != '-:-':
            # determine if game went to overtime or shootout
            overtime = 1 if row['OT/SO'] == 'OT' else 0
            shootout = 1 if row['OT/SO'] == 'SO' else 0

            game_players = player_dic[league_code]
            player_dat = game_players[game_players['game_id'] == game_id]

            # goals and assists by position
            home_goals_forwards = player_dat[(player_dat['level_0'] == 'player_home') & (player_dat['Pos'] == 'Stürmer')]['G'].sum().astype(int)
            away_goals_forwards = player_dat[(player_dat['level_0'] == 'player_away') & (player_dat['Pos'] == 'Stürmer')]['G'].sum().astype(int)
            home_goals_defense = player_dat[(player_dat['level_0'] == 'player_home') & (player_dat['Pos'] == 'Verteidiger')]['G'].sum().astype(int)
            away_goals_defense = player_dat[(player_dat['level_0'] == 'player_away') & (player_dat['Pos'] == 'Verteidiger')]['G'].sum().astype(int)

            home_assists_forwards = player_dat[(player_dat['level_0'] == 'player_home') & (player_dat['Pos'] == 'Stürmer')]['A'].sum().astype(int)
            away_assists_forwards = player_dat[(player_dat['level_0'] == 'player_away') & (player_dat['Pos'] == 'Stürmer')]['A'].sum().astype(int)
            home_assists_defense = player_dat[(player_dat['level_0'] == 'player_home') & (player_dat['Pos'] == 'Verteidiger')]['A'].sum().astype(int)
            away_assists_defense = player_dat[(player_dat['level_0'] == 'player_away') & (player_dat['Pos'] == 'Verteidiger')]['A'].sum().astype(int)

            # penalty minutes
            game_stats = game_stats_dic[league_code]
            game_dat = game_stats[game_stats['game_id'] == game_id]
            home_penalty_minutes = game_dat[(game_dat['Stats'] == 'PIM Total')]['Home']
            try:
                home_penalty_minutes = float(home_penalty_minutes.iloc[0])
            except:
                home_penalty_minutes = float(0)
            away_penalty_minutes = game_dat[(game_dat['Stats'] == 'PIM Total')]['Away']
            try:
                away_penalty_minutes = float(away_penalty_minutes.iloc[0])
            except:
                away_penalty_minutes = float(0)

            # special teams stats
            home_powerplay_n = int(game_dat[(game_dat['Stats'] == 'PP OP')]['Home'].iloc[0])
            away_powerplay_n = int(game_dat[(game_dat['Stats'] == 'PP OP')]['Away'].iloc[0])

            home_powerplay_goals = int(game_dat[(game_dat['Stats'] == 'PPG')]['Home'].iloc[0])
            away_powerplay_goals = int(game_dat[(game_dat['Stats'] == 'PPG')]['Away'].iloc[0])

            home_shorthand_goals = int(game_dat[(game_dat['Stats'] == 'SHG')]['Home'].iloc[0])
            away_shorthand_goals = int(game_dat[(game_dat['Stats'] == 'SHG')]['Away'].iloc[0])

            home_powerplay_min = game_dat[(game_dat['Stats'] == 'PPT')]['Home']
            home_powerplay_min = float(home_powerplay_min.iloc[0].replace(':','.'))
            away_powerplay_min = game_dat[(game_dat['Stats'] == 'PPT')]['Away']
            away_powerplay_min = float(away_powerplay_min.iloc[0].replace(':','.'))
        
            # Add all data to the dataset
            game_info.loc[(i+1)] = {
                "id": game_id,
                "Season": row['level_0'],
                "Region": row['Region'],
                "Phase": row['Phase'],
                "Date": row['Datum'],
                "Weekday": row['Tag'],
                "Time": row['Zeit'],
                "Spectators": row['Zus.'],
                "Home_Team": row['Home'],
                "Away_Team": row['Away'],
                "Result": row['Resultat'],
                "Period_Results": row['Drittel'],
                "OT": overtime,
                "SO": shootout,
                "Home_Goals_Forwards": home_goals_forwards,
                "Away_Goals_Forwards": away_goals_forwards,
                "Home_Goals_Defense": home_goals_defense,
                "Away_Goals_Defense": away_goals_defense,
                "Home_Assists_Forwards": home_assists_forwards,
                "Away_Assists_Forwards": away_assists_forwards,
                "Home_Assists_Defense": home_assists_defense,
                "Away_Assists_Defense": away_assists_defense,
                "Home_Penalty_Min": home_penalty_minutes,
                "Away_Penalty_Min": away_penalty_minutes,
                "Home_Powerplay_Opportunities": home_powerplay_n,
                "Away_Powerplay_Opportunities": away_powerplay_n,
                "Home_Powerplay_Goals": home_powerplay_goals,
                "Away_Powerplay_Goals": away_powerplay_goals,
                "Home_Shorthanded_Goals": home_shorthand_goals,
                "Away_Shorthanded_Goals": away_shorthand_goals,
                "Home_Powerplay_Min": home_powerplay_min,
                "Away_Powerplay_Min": away_powerplay_min
            }

    # Save data to SQLite database
    conn = db.connect('../data/hockey_stats.db')
    game_info.to_sql(f'{league_code}_game_stats', conn, if_exists='replace', index=False)
    conn.commit()
    conn.close()

In [249]:
# Clean up games dataframe for saving
for league_name, league_code in league_dic.items():
    games = games_dic[league_code]
    games = games.copy()
    games = games[['Id', 'level_0', 'Region', 'Phase', 'Datum', 'Tag', 'Zeit', 'Home', 'Away', 'Resultat']]
    games.rename(columns={'level_0': 'Season', 'Datum': 'Date', 'Tag': 'Weekday', 'Zeit': 'Time'}, inplace=True)

    # Save data to SQLite database
    conn = db.connect('../data/hockey_stats.db')
    games.to_sql(f'{league_code}_games', conn, if_exists='replace', index=False)
    conn.commit()
    conn.close()

## Players per Game

In [250]:
# get and save table of players per game
for league_name, league_code in league_dic.items():
    players_game = player_dic[league_code]
    players_game = players_game.copy()
    players_game.drop(columns=['level_1', 'A1', 'A2', 'PTS'], inplace=True)
    players_game.rename(columns={'level_0': 'home_away', 'No': 'number', 'Spieler': 'player_name',
                                    'Pos': 'position', 'G': 'goals', 'A': 'assists',
                                    'PIM Total': 'penalty_minutes'}, inplace=True)
    players_game['number'] = players_game['number'].astype(int)
    players_game['goals'] = players_game['goals'].astype(int)
    players_game['assists'] = players_game['assists'].astype(int)
    players_game['penalty_minutes'] = players_game['penalty_minutes'].astype(int)

    players_game = players_game[['game_id', 'home_away', 'number', 'player_name', 'position', 'goals', 'assists', 'penalty_minutes']]

    # Save data to SQLite database
    conn = db.connect('../data/hockey_stats.db')
    players_game.to_sql(f'{league_code}_players_per_game', conn, if_exists='replace', index=False)
    conn.commit()
    conn.close()


## Goalies per Game

In [ ]:
# get and save table of goalies per game
for league_name, league_code in league_dic.items():
    goalies_game = goalie_dic[league_code]
    goalies_game = goalies_game.copy()
    goalies_game.drop(columns=['level_1'], inplace=True)
    goalies_game.rename(columns={'level_0': 'home_away', 'No': 'number', 'Spieler': 'goalie_name',
                                    'GA': 'goals_allowed', 'MIP': 'minutes_played', 'GAA': 'goals_allowed_game',
                                    'PIM': 'penalty_minutes'}, inplace=True)
    goalies_game['number'] = goalies_game['number'].astype(int)
    goalies_game['goals_allowed'] = goalies_game['goals_allowed'].astype(int)
    goalies_game['minutes_played'] = goalies_game['minutes_played'].astype(int)
    goalies_game['penalty_minutes'] = goalies_game['penalty_minutes'].astype(int)
    goalies_game['goals_allowed_game'] = goalies_game['goals_allowed_game'].replace('-', '0')
    goalies_game['goals_allowed_game'] = goalies_game['goals_allowed_game'].astype(float)

    goalies_game = goalies_game[['game_id', 'home_away', 'number', 'goalie_name', 'goals_allowed', 'minutes_played',
                                     'goals_allowed_game', 'penalty_minutes']]

    # Save data to SQLite database
    conn = db.connect('../data/hockey_stats.db')
    goalies_game.to_sql(f'{league_code}_goalies_per_game', conn, if_exists='replace', index=False)
    conn.commit()
    conn.close()

C:\Users\bwu\AppData\Local\Temp\ipykernel_9684\998353457.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  goalies_game['goals_allowed_game'] = goalies_game['goals_allowed_game'].replace('-', '0', inplace=True)
C:\Users\bwu\AppData\Local\Temp\ipykernel_9684\998353457.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are 